In [1]:
import tensorflow as tf
import numpy as np
print("TensorFlow version: ",format(tf.__version__))
from pathlib import Path
import DataSets as ds

TensorFlow version:  2.0.0


### chargement des données

In [2]:
LoadModel = True
experiment_name = '1k'
train = ds.DataSet(Path("/home/romain/jupyter_mla/Databases/data_1k.bin"), \
                   Path("/home/romain/jupyter_mla/Databases/gender_1k.bin"), 1000)

nb data =  1000


### classes pour définir les couches et la structure du réseau

Note: le module tf.summary est une API qui permet d'enregristrer des mesures de données. Ces enregistrements peuvent ensuite être visualisés sous TensorBoard, l'outil de visualisation de TensorFlow.

In [3]:
class fc_layer(tf.Module):
    def __init__(self, name, input_dim, output_dim):
        self.scope_name = name
        w_init = tf.random.truncated_normal([input_dim, output_dim], stddev=0.1)
        self.w = tf.Variable(w_init)
        print('w      ', self.w.shape)
        b_init = tf.constant(0.0, shape=[output_dim])
        self.b = tf.Variable(b_init)
        print('b      ', self.b.get_shape())

    def __call__(self, x, log_summary):
        if log_summary:
            with tf.name_scope(self.scope_name) as scope:
                tf.summary.scalar("mean w", tf.reduce_mean(self.w))
                tf.summary.scalar("max w", tf.reduce_max(self.w))
                tf.summary.histogram("w", self.w)
                tf.summary.scalar("mean b", tf.reduce_mean(self.b))
                tf.summary.scalar("max b", tf.reduce_max(self.b))
                tf.summary.histogram("b", self.b)
        return tf.matmul(x, self.w) + self.b

In [4]:
class SimpleNet(tf.Module):
    def __init__(self, input_dim):
        self.fc1 = fc_layer('fc1', input_dim, 50)
        self.fc2 = fc_layer('fc2', 50, 2)

    def __call__(self, x, log_summary):
        x = self.fc1(x, log_summary)
        x = tf.nn.sigmoid(x)
        x = self.fc2(x, log_summary)
        return x

In [5]:
def train_one_iter(model, optimizer, image, label, log_summary):
    with tf.GradientTape() as tape:
        y = model(image,log_summary)
        loss = tf.reduce_sum(tf.square(y - label))
        if log_summary:
            tf.summary.scalar('loss', loss)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

### Résumé du modèle

In [6]:
print ("-----------------------------------------------------")
print ("-----------", experiment_name, "---------------------")
print ("-----------------------------------------------------")

-----------------------------------------------------
----------- 1k ---------------------
-----------------------------------------------------


In [7]:
# note: 'tf.summary.create_file_writer' sert à créer un fichier récapitulatif dans le répertoire en argument
train_summary_writer = tf.summary.create_file_writer('logs')
optimizer = tf.optimizers.SGD(1e-4)
simple_v2 = SimpleNet(train.dim)

w       (2304, 50)
b       (50,)
w       (50, 2)
b       (2,)


In [8]:
# note: 'tf.summary.Checkpoint' sert à créer un point de sauvegarde et de restauration des paramètres suivis
if LoadModel:
    ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=simple_v2)
    ckpt.restore('/home/romain/jupyter_mla/Databases/saved_model-1')

In [9]:
for iter in range(500):
    tf.summary.experimental.set_step(iter)
    ima, lab = train.NextTrainingBatch()
    with train_summary_writer.as_default():
        loss = train_one_iter(simple_v2, optimizer, ima, lab, iter % 10 == 0)
    if iter % 100 == 0:
        print("iter= %6d - loss= %f" % (iter, loss))

iter=      0 - loss= 42.924492
iter=    100 - loss= 36.503040
iter=    200 - loss= 22.528732
iter=    300 - loss= 40.310223
iter=    400 - loss= 33.822441


In [10]:
if not LoadModel:
    ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=simple_v2)
    ckpt.save(Path("/home/romain/jupyter_mla/Databases/saved_model"))